In [ ]:
print("Hello, World")

Hello, World


In [6]:
#Forearm
import cv2
import mediapipe as mp
import socket
import numpy as np

def calculate_angle(p1, p2, p3):
    v1 = np.array(p2) - np.array(p1)
    v2 = np.array(p3) - np.array(p1)
    
    v1_u = v1 / np.linalg.norm(v1)
    v2_u = v2 / np.linalg.norm(v2)

    dot_product = np.dot(v1_u, v2_u)
    angle_rad = np.arccos(dot_product)
    angle_deg = np.degrees(angle_rad)
    
    return angle_deg

sock = socket.socket(socket.AF_INET,socket.SOCK_DGRAM) 
serverAddressPort = ("127.0.0.1",69)

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()


cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  

    result = hands.process(frame)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # for landmark in hand_landmarks.landmark:
            #     landmark_px = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
            #     cv2.circle(frame, landmark_px, 5, (255, 0, 0), -1)
                
            thumb_tip = [hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x, hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y]

            pt = [hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x, hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y]
            
            wrist_px = int(pt[0] * frame.shape[1]), int(pt[1] * frame.shape[0])
            wrist_px2 = (wrist_px[0], wrist_px[1] - 100)
            thumb_pt = int(thumb_tip[0] * frame.shape[1]), int(thumb_tip[1] * frame.shape[0])
            
            cv2.line(frame, wrist_px, wrist_px2, (0, 255, 255), 2)
            cv2.line(frame, wrist_px, thumb_pt, (0, 255, 255), 2)
            
            angle = calculate_angle(wrist_px, wrist_px2, thumb_pt)

            if (angle < 45):
                # print('not', f"{angle:.2f}")
                sock.sendto(str.encode(f'0,{angle:.2f}'),serverAddressPort)
            elif thumb_tip[0] < pt[0]: #Right
                # print('d', f"{angle:.2f}")
                sock.sendto(str.encode(f'1,{angle:.2f}'),serverAddressPort)
            elif thumb_tip[0] > pt[0]:
                # print('u', f"{angle:.2f}")
                sock.sendto(str.encode(f'-1,{angle:.2f}'),serverAddressPort)
            # else:
            #     sock.sendto(str.encode('0'),serverAddressPort)

    cv2.imshow('MediaPipe Pose', frame)
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

In [3]:
#Wrist For RightHand
import cv2
import mediapipe as mp
import socket
import numpy as np

def calculate_angle(p1, p2, p3):
    v1 = np.array(p2) - np.array(p1)
    v2 = np.array(p3) - np.array(p1)
    
    v1_u = v1 / np.linalg.norm(v1)
    v2_u = v2 / np.linalg.norm(v2)

    dot_product = np.dot(v1_u, v2_u)
    angle_rad = np.arccos(dot_product)
    angle_deg = np.degrees(angle_rad)
    
    return angle_deg

sock = socket.socket(socket.AF_INET,socket.SOCK_DGRAM) 
serverAddressPort = ("127.0.0.1",69)

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

RightThreshold = 0 
LeftThreshold = 0
tipset1 = [5] #for rt, index knuckle
tipset2 = [17] #for left, pinky knuckle
#the finger tips used are to promote the most amount of motion. But if unable to do it by a patient. then compare wrist to the finger tips (index or pinky). Now using knuckles.
#and if unable to do even finger tips. then move to the next finger

temp = 0
temp2 = 0
temp3 = 0

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  

    result = hands.process(frame)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            for landmark in hand_landmarks.landmark:
                landmark_px = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                cv2.circle(frame, landmark_px, 5, (255, 0, 0), -1)
        
            pt = [hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x, hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y]
            
            wrist_px = int(pt[0] * frame.shape[1]), int(pt[1] * frame.shape[0])
            wrist_px2 = (wrist_px[0], wrist_px[1] - 100)
            
            MiddleTwo_tip = [(hand_landmarks.landmark[12].x + hand_landmarks.landmark[16].x)/2, (hand_landmarks.landmark[12].y + hand_landmarks.landmark[16].y)/2]
            MiddleTwo_pt = int(MiddleTwo_tip[0] * frame.shape[1]), int(MiddleTwo_tip[1] * frame.shape[0])
            
            cv2.line(frame, wrist_px, wrist_px2, (0, 255, 255), 2)
            cv2.line(frame, wrist_px, MiddleTwo_pt, (0, 255, 255), 2)
            
            angle = calculate_angle(wrist_px, wrist_px2, MiddleTwo_pt)
            
            # print(pt[0], hand_landmarks.landmark[tipset2[0]].x)
            if(pt[0] > hand_landmarks.landmark[tipset1[0]].x + RightThreshold):
                # temp += 1
                # print("right"+ str(temp), angle, pt[0] , hand_landmarks.landmark[tipset1[0]].x)
                sock.sendto(str.encode(f'1,{angle:.2f}'),serverAddressPort)
            elif(pt[0] < hand_landmarks.landmark[tipset2[0]].x + LeftThreshold):
                # temp2 += 1
                # print("left" + str(temp2), angle)
                sock.sendto(str.encode(f'-1,{angle:.2f}'),serverAddressPort)
            else:
                # temp3 += 1
                # print("None" + str(temp3), angle)
                sock.sendto(str.encode(f'0,{angle:.2f}'),serverAddressPort)

    cv2.imshow('MediaPipe Pose', frame)
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()